In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\AI-detector'

In [12]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Vinay_Gupta/AI-detector.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Vinay_Gupta"
os.environ["MLFLOW_TRACKING_PASSWORD"]="fb955b2fdcfdc10bac93954932677b94948428e5"

In [13]:
import tensorflow as tf

In [14]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluateConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [16]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json


In [17]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    def get_evaluation_config(self) -> EvaluateConfig:
          eval_config=EvaluateConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
                mlflow_uri="https://dagshub.com/Vinay_Gupta/AI-detector.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )
          return eval_config

In [18]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [19]:
class Evaluation:
    def __init__(self,config: EvaluateConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_score=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],"accuracy":self.score[1]
            })

            if tracking_uri_type_score!="file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
    

In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 102 images belonging to 2 classes.


d:\AI-detector\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 775ms/step - accuracy: 0.2034 - loss: 31.9330


2024/12/28 22:22:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/12/28 22:23:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2024/12/28 22:25:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run fun-lynx-816 at: https://dagshub.com/Vinay_Gupta/AI-detector.mlflow/#/experiments/0/runs/5b0bd2cc6fe1438d9f211b7a94996441
🧪 View experiment at: https://dagshub.com/Vinay_Gupta/AI-detector.mlflow/#/experiments/0
